# Physical Object: River Channel

The `RiverChannel` class models a segment of a river using a simple yet effective **linear reservoir model**. In this model, the outflow from the river channel is directly proportional to the volume of water currently stored in it.

This type of model is excellent for capturing the storage and attenuation effects of a river reach, where a flood wave entering the channel will be smoothed and delayed as it travels downstream.

### State Variables
- `volume` (float): The current volume of water in the channel (m^3).
- `outflow` (float): The calculated outflow, proportional to the volume (m^3/s).

### Parameters
- `k` (float): The storage coefficient (1/s). A smaller `k` means a longer residence time and more attenuation.

## Simulation Example

The simulation below demonstrates the attenuation effect of the river channel. We will send a sharp, pulse-like inflow (a simple 'flood wave') into the channel and observe how the outflow is lower and spread out over a longer period.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from swp.simulation_identification.physical_objects.river_channel import RiverChannel
from swp.core.interfaces import State, Parameters

# 1. Define initial state and parameters
initial_state = State(volume=5e5, outflow=50)
parameters = Parameters(k=0.0001)

# 2. Create a RiverChannel instance
channel = RiverChannel(name="main_reach", initial_state=initial_state, parameters=parameters)

# 3. Simulation settings
dt = 600 # 10-minute time steps
simulation_duration = 86400 * 2 # 2 days
num_steps = int(simulation_duration / dt)
history = []
inflows = []

# 4. Run the simulation loop
base_inflow = 50.0
for t in range(num_steps):
    # Create a flood wave pulse
    if 10 <= t < 30:
        inflow = base_inflow + 500.0
    else:
        inflow = base_inflow
    inflows.append(inflow)
    
    channel.set_inflow(inflow)
    
    # The step action is not used by this model
    current_state = channel.step(action=None, dt=dt)
    history.append(current_state.copy())

print("River channel simulation complete.")

## Results and Visualization

The plot clearly shows that while the inflow is a sharp, square pulse, the outflow is a much smoother, lower-peaked curve. This is the characteristic attenuation behavior of a linear reservoir model.

In [ ]:
# Extract data from history
time_hours = [i * dt / 3600 for i in range(num_steps)]
volumes_Mm3 = [h['volume'] / 1e6 for h in history]
outflows = [h['outflow'] for h in history]

# Create a DataFrame
df = pd.DataFrame({
    'Time (hours)': time_hours,
    'Inflow (m^3/s)': inflows,
    'Outflow (m^3/s)': outflows,
    'Volume (Mm^3)': volumes_Mm3
})

print(df.head())

# Plot the results
fig, ax1 = plt.subplots(figsize=(12, 6))

# Flows
ax1.plot(df['Time (hours)'], df['Inflow (m^3/s)'], label='Inflow', color='blue', linestyle='--')
ax1.plot(df['Time (hours)'], df['Outflow (m^3/s)'], label='Outflow', color='red')
ax1.set_xlabel('Time (hours)')
ax1.set_ylabel('Flow Rate (m^3/s)')
ax1.set_title('River Channel Flood Wave Attenuation')
ax1.grid(True)
ax1.legend()

plt.tight_layout()
plt.show()